# Bead on a hoop

A circular wire hoop rotates with constant angular velocity $\omega$ about a vertical diameter. A small bead
moves, without friction, along the hoop.

The equation of motion can be shown to be (using the standard notation in classical mechanics) :

$$ \ddot{\theta} = -\omega_c^2 \sin \theta + \omega^2 \sin \theta \, \cos \theta - \alpha \theta$$

with $\omega_c = \sqrt{g/R}$, where the gravity acceleration is denoted by g and the radius of the hoop is denoted R. The coefficient $\alpha$ is related to the friction in the system and can be idealized to be zero in the frictionless configuration.

Let $y_1=\theta$ and $y_2 = \dot{\theta}$. Then, we can switch to a first order system of differential equations :

$$
\left\{\begin{aligned}
d_t y_1 & = y_2\\
d_t y_2 & = \sin y1 \, (\omega^2 \cos y_1 - \omega_c^2) - \alpha y_2
\end{aligned}\right.
$$

In the frictionless configuration, the system can be written :
$$
\left\{\begin{aligned}
d_t y_1 & = y_2\\
d_t y_2 & = \sin y1 \, (\omega^2 \cos y_1 - \omega_c^2)
\end{aligned}\right.
$$

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, LabelSet

from ipywidgets import interact, FloatSlider

from mylib.model import bead_hoop_model, bead_hoop_friction_model
import mylib.continuation as continuation

output_notebook(hide_banner=True)

In [2]:
def plot_frictionless_sol():

    yini = (1. , 1.)
    tini = 0.
    tend = 10
    nt = 1001
    
    omega_c = np.sqrt(9.81)
    
    bhm = bead_hoop_model(omega=4.)
    fcn = bhm.fcn 
    jac_eq_1 = bhm.jac_eq_1
    jac_eq_2 = bhm.jac_eq_2
        
    tol = 1.e-10
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    eig_vals_eq_1, _= np.linalg.eig(jac_eq_1())
    eig_vals_eq_2, _= np.linalg.eig(jac_eq_2())
  
    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_y1 = fig_sol.line(sol.t, sol.y[0], legend="y1", line_width=2)    
    plt_y2 = fig_sol.line(sol.t, sol.y[1], legend="y2", line_width=2, color="Green")
    
    fig_pha = figure(plot_height=300, plot_width=475, title="Phase plan")
    plt_pha = fig_pha.line(sol.y[0], sol.y[1], line_width=2)
    
    fig_eig = figure(x_range=(-6,6), y_range=(-3.5,3.5), plot_height=300, plot_width=475, 
                     title="Eigen values at equilibrium points")
    fig_eig.line(x=np.linspace(-6.,6,10), y=0, color="black")
    fig_eig.line(x=0, y=np.linspace(-3.5,3.5,10), color="black")
    plt_eig_1 = fig_eig.circle(np.real(eig_vals_eq_1), np.imag(eig_vals_eq_1), line_width=2, size=10, color="crimson",
                               legend="Unstable values")
    plt_eig_2 = fig_eig.circle(np.real(eig_vals_eq_2), np.imag(eig_vals_eq_2), line_width=2, size=10, color="green",
                               legend="Stable values")


    show(column(fig_sol, row(fig_pha, fig_eig)), notebook_handle=True)
    
    def update(omega):
        bhm = bead_hoop_model(omega)
        fcn = bhm.fcn 
        jac_eq_1 = bhm.jac_eq_1
        jac_eq_2 = bhm.jac_eq_2
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        eig_vals_eq_1, _= np.linalg.eig(jac_eq_1())
        eig_vals_eq_2, _= np.linalg.eig(jac_eq_2())

        plt_y1.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_y2.data_source.data = dict(x=sol.t, y=sol.y[1])
        plt_pha.data_source.data = dict(x=sol.y[0], y=sol.y[1])
        plt_eig_1.data_source.data = dict(x=np.real(eig_vals_eq_1), y=np.imag(eig_vals_eq_1))
        plt_eig_2.data_source.data = dict(x=np.real(eig_vals_eq_2), y=np.imag(eig_vals_eq_2))
        push_notebook()

    min = omega_c - 3.13
    max = omega_c + 3.13
    step = (max-min)/100
    interact(update, omega=FloatSlider(min=min, max=max, value=omega_c, step=step, continious_update=False))

    
plot_frictionless_sol()

interactive(children=(FloatSlider(value=3.132091952673165, description='omega', max=6.262091952673165, min=0.0…

In [3]:
def plot_friction_sol():

    yini = (1. , 1.)
    tini = 0.
    tend = 10
    nt = 1001
    
    omega_c = np.sqrt(9.81)
    alpha = 0.5
    
    bhfm = bead_hoop_friction_model(omega=2., alpha=alpha)
    fcn = bhfm.fcn 
    jac_eq_1 = bhfm.jac_eq_1
    jac_eq_2 = bhfm.jac_eq_2
        
    tol = 1.e-10
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    eig_vals_eq_1, _= np.linalg.eig(jac_eq_1())
    eig_vals_eq_2, _= np.linalg.eig(jac_eq_2())
  
    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_y1 = fig_sol.line(sol.t, sol.y[0], legend="y1", line_width=2)    
    plt_y2 = fig_sol.line(sol.t, sol.y[1], legend="y2", line_width=2, color="Green")
    
    fig_pha = figure(plot_height=300, plot_width=475, title="Phase plan")
    plt_pha = fig_pha.line(sol.y[0], sol.y[1], line_width=2)
    
    fig_eig = figure(x_range=(-6,6), y_range=(-3.5,3.5), plot_height=300, plot_width=475, 
                     title="Eigen values at equilibrium points")
    fig_eig.line(x=np.linspace(-6.,6,10), y=0, color="black")
    fig_eig.line(x=0, y=np.linspace(-3.5,3.5,10), color="black")
    plt_eig_1 = fig_eig.circle(np.real(eig_vals_eq_1), np.imag(eig_vals_eq_1), line_width=2, size=10, color="crimson",
                               legend="Unstable values")
    plt_eig_2 = fig_eig.circle(np.real(eig_vals_eq_2), np.imag(eig_vals_eq_2), line_width=2, size=10, color="green",
                               legend="Stable values")

    show(column(fig_sol, row(fig_pha, fig_eig)), notebook_handle=True)
    
    def update(omega):
        bhfm = bead_hoop_friction_model(omega, alpha=alpha)
        fcn = bhfm.fcn 
        jac_eq_1 = bhfm.jac_eq_1
        jac_eq_2 = bhfm.jac_eq_2
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        eig_vals_eq_1, _= np.linalg.eig(jac_eq_1())
        eig_vals_eq_2, _= np.linalg.eig(jac_eq_2())

        plt_y1.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_y2.data_source.data = dict(x=sol.t, y=sol.y[1])
        plt_pha.data_source.data = dict(x=sol.y[0], y=sol.y[1])
        plt_eig_1.data_source.data = dict(x=np.real(eig_vals_eq_1), y=np.imag(eig_vals_eq_1))
        plt_eig_2.data_source.data = dict(x=np.real(eig_vals_eq_2), y=np.imag(eig_vals_eq_2))
        push_notebook()

    min = omega_c - 3.13
    max = omega_c + 3.13
    step = (max-min)/100
    interact(update, omega=FloatSlider(min=min, max=max, value=omega_c, step=step, continious_update=False))

    
plot_friction_sol()

interactive(children=(FloatSlider(value=3.132091952673165, description='omega', max=6.262091952673165, min=0.0…

In [4]:
def show_continuation():
    
    omega_c = np.sqrt(9.81)

    omega_eq_ini = 0.0
    y1_eq_ini = 0.

    s_ini = 0.0
    s_end = 10.0
    ns = 201

    omega_eq, y1_eq = continuation.pseudo_arclength_continuation_for_hoop(s_ini, s_end, ns, omega_eq_ini, y1_eq_ini)

    fig_branch = figure(plot_height=500, plot_width=950, x_range=(0,10))
    fig_branch.x(omega_eq[:, 0], y1_eq[:, 0], color="Red", legend="unstable branch")
    fig_branch.x(omega_eq[:, 1], y1_eq[:, 1], color="Green", legend="stable branch")
    fig_branch.x(omega_eq[:, 2], y1_eq[:, 2], color="Green", legend="stable branch")
    source = ColumnDataSource(data=dict(x=[omega_c], y=[0], name=["pitchfork bifurcation"]))
    fig_branch.x(x='x', y='y', source=source, size=10, line_width=2, legend="pitchfork bifurcation")
    labels = LabelSet(x='x', y='y', text='name', source=source, x_offset=10, y_offset=10)
    fig_branch.add_layout(labels)

    fig_branch.legend.location = "top_left"
    show(fig_branch)


show_continuation()

* 红色的point equilibre是不再stable了
* 需要在bifurcation定义方向


* 如果用d J/d s 的方法总是走右边的，然而在critique的点的时候稍微调整jacobian
* 在作图的时候最好写出，画出valeur propre
* continuation？？？？？？？？？？？？？？？？？？？？？
